In [32]:
import spacy
from tqdm import tqdm
from utils import load_jsonl_iteratively

In [33]:
import os
from utils import dump_jsonl

ja_sentence_info = "/home/xzhao/workspace/med-eval/dataset/losses/train_samples/ja_1k_ner_with_pos.jsonl"

if not os.path.exists(ja_sentence_info):
    inputs = []
    nlp = spacy.load("ja_core_news_trf")
    for item in tqdm(load_jsonl_iteratively("/home/xzhao/workspace/med-eval/dataset/losses/train_samples/ja_1k_ner.jsonl")):
        doc = nlp(item['text'])
        pos = []
        for token in doc:
            assert item['text'][token.idx: token.idx + len(token.text)] == token.text
            pos.append({
                "text": token.text,
                "span": [token.idx, token.idx + len(token.text)],
                "type": token.pos_,
            })
        item['pos'] = pos
        inputs.append(item)
    dump_jsonl(inputs, "/home/xzhao/workspace/med-eval/dataset/losses/train_samples/ja_1k_ner_with_pos.jsonl")
else:
    inputs = list(load_jsonl_iteratively(ja_sentence_info))

/home/xzhao/softwares/anaconda/envs/llm-eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
1000it [01:03, 15.84it/s]


In [34]:
import os
import ast
from tqdm import tqdm
import re
import pandas as pd

def load_jsonl(filename):
    assert os.path.exists(filename), filename
    with open(filename, 'r', encoding="utf8") as fp:
        fixed_data = ast.literal_eval(fp.read())
    return fixed_data


iterations = []
root = "/home/xzhao/workspace/med-eval/dataset/losses/results/per_token_loss/llm-jp-3-13b-exp1-en_jstage"
for filename in tqdm(sorted(os.listdir(root))):
    if filename.endswith(".json"):
        match = re.match("iter_([0-9]+).json", filename)
        if match:
            iter_num = int(match.group(1))
        else:
            continue
        
        if iter_num >= 1:
            continue
        item = {"iter": iter_num}
        filepath = os.path.join(root, filename)
        data = load_jsonl(filepath)
        iterations.append(data)


100%|██████████| 19/19 [00:09<00:00,  2.11it/s]


In [23]:
def categorize_token(token: str) -> str:
    """Categorize a token as 'Japanese', 'English', or 'Other'."""
    if all('ぁ' <= char <= 'ん' or   # Hiragana
           'ァ' <= char <= 'ン' or   # Katakana
           '一' <= char <= '龯'      # Kanji
           for char in token if char.strip()):
        return 'Japanese'
    elif all(char.isascii() and char.isalpha() for char in token):
        return 'English'
    else:
        return 'En-Punctuation' if all(char in '.,;:!?()[]{}' for char in token) else 'Ja-Punctuation'

In [36]:
unparsable_cnt = 0
for item in tqdm(inputs):
    text = item['text']
    assert text in iterations[0]['ja']['losses']
    item['token'] = []
    tokens, token_ids, token_spans, langs, losses = [], [], [], [], []
    search_start = 0
    for idx, (token, token_id, _) in enumerate(iterations[0]['ja']['losses'][item['text']]['token_losses']):
        try:
            start = text.index(token, search_start)
            end = start + len(token)
            search_start = end
            assert text[start:end] == token, f"Token mismatch: {text[start:end]} != {token}"
            
            tokens.append(token)
            token_ids.append(token_id)
            token_spans.append([start, end])
            langs.append(categorize_token(token))
        except ValueError:
            unparsable_cnt += 1
            continue
    
    for token, token_id, token_span, lang in zip(tokens, token_ids, token_spans, langs):
        item['token'].append({
            "text": token,
            "span": token_span,
            "type": lang,
            "token_id": token_id
        })
print(f"Unparsable tokens count: {unparsable_cnt}")
dump_jsonl(inputs, ja_sentence_info)

100%|██████████| 1000/1000 [00:00<00:00, 3733.82it/s]


Unparsable tokens count: 162


In [67]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-3-7.2b")


In [38]:
inputs[0]['token'][:10]  # Display first 10 tokens for verification

[{'text': '二', 'span': [0, 1], 'type': 'Japanese', 'token_id': 46171},
 {'text': 'つ', 'span': [1, 2], 'type': 'Japanese', 'token_id': 30077},
 {'text': '目', 'span': [2, 3], 'type': 'Japanese', 'token_id': 29227},
 {'text': 'の', 'span': [3, 4], 'type': 'Japanese', 'token_id': 28998},
 {'text': '文', 'span': [4, 5], 'type': 'Japanese', 'token_id': 29024},
 {'text': 'は', 'span': [5, 6], 'type': 'Japanese', 'token_id': 29282},
 {'text': '一つ', 'span': [6, 8], 'type': 'Japanese', 'token_id': 64911},
 {'text': '目', 'span': [8, 9], 'type': 'Japanese', 'token_id': 29227},
 {'text': 'の', 'span': [9, 10], 'type': 'Japanese', 'token_id': 28998},
 {'text': '文', 'span': [10, 11], 'type': 'Japanese', 'token_id': 29024}]

In [4]:
text = texts[1]
print(text)
loss1 = items[0]["ja"]['losses'][text]
loss2 = items[1]["ja"]['losses'][text]
loss3 = items[2]["ja"]['losses'][text]
loss4 = items[3]["ja"]['losses'][text]
loss5 = items[4]["ja"]['losses'][text]
loss6 = items[5]["ja"]['losses'][text]
loss7 = items[6]["ja"]['losses'][text]


次の文章の最初の部分をもとに、続きを書いてください。
生前に両側性の腎腫大が認められた猫において, 臨床的に尿毒症, 呼吸困難, 片側性の瞳孔散大が認められた. 肉眼的に両腎における腫瘍性腫瘤ならびに肺にその転移巣と思われる腫瘤が多数認められた. 病理組織検索を行った結果, 腎腫瘤は腎癌と診断された....
また肺, 心臓, 動眼神経線維東および乳頭体への腫瘍転移巣が認められた, 動眼神経の一部に存在する動眼神経副交感神経核からの遠心性線維は瞳孔調節に関与していることから, 本例の瞳孔散大は腎癌の転移による動眼神経の傷害に起因すると推察された.


In [ ]:
doc = nlp(text)
for token in doc:
    print(token.text, token.pos_)

In [19]:
for item1, item2, item3, item4, item5, item6, item7 in zip(loss1['token_losses'], loss2['token_losses'], loss3['token_losses'], loss4['token_losses'], loss5['token_losses'], loss6['token_losses'], loss7['token_losses']):
    print(f"""{item1[0]}\t{item1[2]:7.4f}, {item2[2]:7.4f}, {item3[2]:7.4f}, {item4[2]:7.4f}, {item5[2]:7.4f}, {item6[2]:7.4f}, {item7[2]:7.4f}""")

次の	 9.9262,  9.5515,  9.3997,  9.3007,  9.2820,  9.3304,  9.4072
文章	 8.3630,  8.0780,  8.0083,  7.8710,  7.7708,  7.7276,  7.6938
の	 1.5666,  1.3303,  1.2553,  1.0992,  1.0582,  1.0440,  1.0536
最初	 6.1993,  6.3742,  6.4027,  6.4799,  6.5083,  6.5335,  6.6032
の	 0.3227,  0.4315,  0.4535,  0.4028,  0.3394,  0.2706,  0.2348
部分	 3.4144,  2.8463,  2.8236,  3.0526,  3.2533,  3.4372,  3.4716
をもとに	 8.9705,  9.2792,  9.4678,  9.3397,  9.4653,  9.5188,  9.5441
、	 0.9173,  0.7780,  0.7197,  0.3455,  0.2651,  0.2631,  0.2809
続き	 2.9882,  2.1896,  2.1949,  2.2319,  2.0887,  1.8710,  1.8596
を	 0.0908,  0.0453,  0.0337,  0.0042,  0.0018,  0.0013,  0.0010
書いて	 2.5319,  3.1252,  3.1444,  2.9558,  3.1871,  3.3584,  3.4437
ください。	 0.6018,  0.2812,  0.2529,  0.2765,  0.2453,  0.2158,  0.1846

	 0.2721,  0.7390,  0.4889,  0.0985,  0.0644,  0.0575,  0.0553
生	10.0571,  8.5452,  8.2988,  8.6064,  8.4469,  8.2976,  8.2532
前に	 4.4428,  4.5763,  4.4907,  4.2997,  4.2086,  4.1675,  4.1660
両側	13.7252, 11.6515, 10.1